# Breast Cancer Wisconsin (Diagnostic)

## 前提知識の説明

### SVMとは

SVMとは分類と回帰の両方に利用可能な教師あり学習のモデルです。  
2次元の特徴量を持つデータに対し、線形モデルによって2クラス分類を行うことを考えます。  
学習データのプロットされた平面上に直線を引いて領域を2つに分ける作業で、分類とは線を引くことであり、SVMはうまい具合に線を引く方法です。  
SVMではどの点にも被らないようにできるだけ広い帯を引こうとします。
- メリット  
他のモデルと比較して高い識別性能が得られる  
- デメリット  
データの前処理やパラメーターの調整、結果の解釈が難しい  
- 特徴
「マージン」という「距離のような概念」の最大化を目的としている。  
平均や分散を使わないので新しいデータが入ってきても全体の再計算は不要。  
線形分離が不可能な場合は、「非線形変換を施したうえでより高次元特徴空間に写像」することで対応できる。

### SVMにおけるカーネルとは

線形分離ができない場合、データをある関数により、より高次元空間に埋め込むことで、SVMを使って線形分離にすることができる。このときに使う関数をカーネルといいます。  
SVMには以下のカーネルが用意されています。  
- linear（線形カーネル）
- poly（多項式カーネル）
- rbf（RBFカーネル）
- sigmoid（シグモイドカーネル）
- precomputed（事前に計算したものを利用する）

### 線形回帰やロジスティック回帰との比較

線形回帰やロジスティック回帰は、目的関数を最小化するために各パラメータで偏微分した式を最急降下法の式に導入しました。  
一方、SVMでは最急降下法は使うが、目的関数の最小値を求める際にベクトルの内積を使います。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## データ・セットの用意
UCIのBreast Cancer Wisconsin (Diagnostic) Data Setを使用  
[breast-cancer-wisconsin.data](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)

# EDA

## Introduction

このデータ分析レポートでは、Breast Cancer Wisconsin (Diagnostic)のデータセットを使用して、SVMアルゴリズムによるモデルを構築し、乳がんかそうではないかを分類するために、データ探索を視覚的に行ないます。

 #### 各特徴量はUCIのBreast Cancer Wisconsin (Diagnostic) Data Setの公式HP内の記載内容を参照します。
>http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29  
1）ID番号  
2）診断（M =悪性、B =良性）  
3-32）  
10個の実数値の特徴量が各セルの核について計算されます。  
a）半径（中心から周囲の点までの距離の平均）  
b）テクスチャ（グレースケール値の標準偏差）  
c）周囲  
d）面積  
e）滑らかさ（半径の長さの局所的変化）  
f）コンパクト性（周囲^ 2 /面積-1.0）  
g）凹み（輪郭の凹部の程度）  
h）凹点（輪郭の凹部の数）  
i）対称性  
j）フラクタル次元（「海岸線近似」-1）  

In [ ]:
df= pd.read_csv("input/bcwd.txt",header=None)
df.head()

カラム名を下記のURLを参照して数字の代わりに文字列を埋めていきます。    >https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/data

In [ ]:
df=df.rename(columns={0:'id',1: 'diagnosis',2: 'radius_mean', 3:'texture_mean', 4:'perimeter_mean',
       5:'area_mean', 6:'smoothness_mean',7: 'compactness_mean',8: 'concavity_mean',
       9:'concave points_mean',10: 'symmetry_mean', 11:'fractal_dimension_mean',
       12:'radius_se', 13:'texture_se',14: 'perimeter_se',15: 'area_se',16: 'smoothness_se',
      17: 'compactness_se',18: 'concavity_se',19: 'concave points_se',20: 'symmetry_se',
       21:'fractal_dimension_se',22: 'radius_worst',23: 'texture_worst',
       24:'perimeter_worst', 25:'area_worst',26: 'smoothness_worst',
       27:'compactness_worst',28: 'concavity_worst',29: 'concave points_worst',
      30: 'symmetry_worst',31: 'fractal_dimension_worst',})
df.head()

2列目はラベルになります。

In [ ]:
df.describe()

In [ ]:
df.info()

カラム数は32（ID、診断、特徴量数：30）  
IDが整数値、診断はカテゴリカルデータ、特徴量は小数値  
データ数は569  
欠損値なし  

idは診断結果と関係がないため削除します。

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df_y=df.diagnosis 
ax=sns.countplot(df_y,label='Count')
B,M =df_y.value_counts()
print('良性の人数: ',B)
print('悪性の人数: ',M)

診断（M =悪性、B =良性）
良性357例、悪性212例

次に腫瘍の半径が診断結果（MとB）にどのように関係しているのかヒストグラムでプロットして確認します。平均値と最悪（最大値）値をそれぞれ表示しています。
半径の数値が小さい方にBの人数が固まっていて、反対に半径大きくなるほどMの人数が増えていき、ある値からはMのみになります。半径は診断結果への影響が大きいと考えられます。  
また平均値より最悪（最大値）値の方が顕著に傾向が表れています。

In [ ]:
f,ax = plt.subplots(figsize=(10,6))
m = plt.hist(df[df["diagnosis"] == "M"].radius_mean,bins=30,fc = (1,0,0,0.5),label = "Malignant")
b = plt.hist(df[df["diagnosis"] == "B"].radius_mean,bins=30,fc = (0,1,0,0.5),label = "Bening")
plt.legend()
plt.xlabel("Radius Mean Values")
plt.ylabel("Frequency")
plt.title("Histogram of Radius Mean for Bening and Malignant Tumors")
plt.show()
frequent_malignant_radius_mean = m[0].max()
index_frequent_malignant_radius_mean = list(m[0]).index(frequent_malignant_radius_mean)
most_frequent_malignant_radius_mean = m[1][index_frequent_malignant_radius_mean]
print("Most frequent malignant radius mean is: ",most_frequent_malignant_radius_mean)

In [ ]:
f,ax = plt.subplots(figsize=(10,6))
m = plt.hist(df[df["diagnosis"] == "M"].radius_worst,bins=30,fc = (1,0,0,0.5),label = "Malignant")
b = plt.hist(df[df["diagnosis"] == "B"].radius_worst,bins=30,fc = (0,1,0,0.5),label = "Bening")
plt.legend()
plt.xlabel("Radius Worst Values")
plt.ylabel("Frequency")
plt.title("Histogram of Radius Worst for Bening and Malignant Tumors")
plt.show()
frequent_malignant_radius_worst = m[0].max()
index_frequent_malignant_radius_worst = list(m[0]).index(frequent_malignant_radius_worst)
most_frequent_malignant_radius_worst = m[1][index_frequent_malignant_radius_worst]
print("Most frequent malignant radius worst is: ",most_frequent_malignant_radius_worst)

次は診断結果と腫瘍部分の面積の平均値area_meanとの関係性をヒストグラムで確認します。

In [ ]:
f,ax = plt.subplots(figsize=(10,6))
m = plt.hist(df[df["diagnosis"] == "M"].area_mean,bins=30,fc = (1,0,0,0.5),label = "Malignant")
b = plt.hist(df[df["diagnosis"] == "B"].area_mean,bins=30,fc = (0,1,0,0.5),label = "Bening")
plt.legend()
plt.xlabel("Area Mean Values")
plt.ylabel("Frequency")
plt.title("Histogram of Area Mean for Bening and Malignant Tumors")
plt.show()
frequent_malignant_area_mean = m[0].max()
index_frequent_malignant_area_mean = list(m[0]).index(frequent_malignant_area_mean)
most_frequent_malignant_area_mean = m[1][index_frequent_malignant_area_mean]
print("Most frequent malignant radius mean is: ",most_frequent_malignant_area_mean)

つぎに3つ以上特徴量比較をするため、ペアグリッドプロットを使用します。  'radius_worst'、'perimeter_worst'と'area_worst'はいずれもworstであるため、ペアのグリッドプロットで可視化すると、当然ながら相関していることが確認できます。

In [ ]:
sns.set(style='whitegrid')
x=df.loc[:,['radius_worst','perimeter_worst','area_worst']]
g=sns.PairGrid(x,diag_sharey=False)
g.map_lower(sns.kdeplot,cmap='Blues_d')
g.map_upper(plt.scatter,edgecolor="w")
g.map_diag(sns.kdeplot,lw=3)

# 前処理の実装と説明

IDのカラムを取得

### カテゴリカルデータの数値化

診断結果のカラムを取得  
診断（M =悪性、B =良性）をmapメソッドで数値化する

In [ ]:
df_y=df.diagnosis 
mapping={'M':1,'B':0}
df_y=df_y.map(mapping)
df_y=pd.DataFrame(df_y)

### 特徴量のみのデータセットを取得
モデルで学習させるためにデータセットから目的変数を取り除いた特徴量のみのデータを作成。

In [ ]:
df_x=df.drop(['diagnosis'],axis=1)
df_x.head()

### 標準化
特徴量の標準化をする。

In [ ]:
cols=df_x.columns

In [ ]:
#標準化：(df_x-df_x.mean())/np.std(df_x)
#標準化を行うコードを記述
from sklearn.preprocessing import StandardScaler

for i in cols:
    x_scaler = StandardScaler()
    x_scaler.fit(df_x[i][:,np.newaxis])
    df_x[i]=x_scaler.transform(df_x[i][:,np.newaxis])

df_x=pd.DataFrame(df_x)
df_x.head()

### データセットの分割
testデータとtrainデータに分割

In [ ]:
from sklearn.model_selection import train_test_split
# test_size：デフォルトは0.25
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,test_size=0.2, random_state=0) 


#### X_trainのshapeを確認

In [ ]:
X_train.shape

#### X_testのshapeを確認

In [ ]:
X_test.shape

# モデルの作成

## チューニング
交差検証を用いたグリッドサーチ

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

tuned_parameters = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
    ]

score = 'recall'
clf = GridSearchCV(SVC(), #識別器
                   tuned_parameters, #最適化したいパラメータセット
                   scoring= '%s_weighted' % score,
                   cv=5, # 交差検証の回数
                   ) 

In [ ]:
clf.fit(X_train, y_train.as_matrix().reshape(-1,))

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

# Accuracy、Recall、Precision、F1-measure

In [ ]:
svm = SVC(gamma=0.001, C=100, probability=True,kernel='rbf')
svm.fit(X_train, y_train)

In [ ]:
y_pred = svm.predict(X_test) 
y_pred

## スクラッチでAccuracyを算出

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_test = y_test.reset_index().drop("index", axis=1)

test_pred=pd.concat([y_test, y_pred], axis=1)
test_pred=test_pred.rename(columns={"diagnosis":"test", 0:"pred"})

In [ ]:
test_pred.head()

In [ ]:
TP=len(test_pred[(test_pred['test'] == 1) & (test_pred['pred'] == 1)])
FN=len(test_pred[(test_pred['test'] == 1) & (test_pred['pred'] == 0)])
FP=len(test_pred[(test_pred['test'] == 0) & (test_pred['pred'] == 1)])
TN=len(test_pred[(test_pred['test'] == 0) & (test_pred['pred'] == 0)])

In [ ]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
accuracy

## sklearn ライブラリーを使ってAccuracyを算出

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

## スクラッチでRecall（検出率）

In [ ]:
recall = TP / (TP + FN)
recall


## sklearn ライブラリーを使って Recall算出

In [ ]:
from sklearn.metrics import recall_score
recall=recall_score(y_test, y_pred)
recall

## スクラッチでPrecision（精度）

In [ ]:
precision = TP / (TP + FP)
precision 

# sklearn ライブラリーを使ってPrecision算出

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)

## スクラッチでF1-measure（F1値）

In [ ]:
f1 = 2 * (precision * recall) / (precision + recall)
f1

## sklearn ライブラリーを使って F1算出

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred) 

## ROC曲線　ライブラリ
http://www.randpy.tokyo/entry/roc_auc

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.svm import SVC

clf =SVC(kernel='rbf', gamma=0.001, C=100,probability=True).fit(X_train, y_train)

prob = clf.predict_proba(X_test)[:,1]
fpr, tpr, thresholds= roc_curve(y_test, prob)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr)
plt.title("ROC curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

 ## AUC　ライブラリ

In [ ]:
auc = np.trapz(tpr,fpr)
print('AUC：'+str(auc))

 ## ROC曲線　スクラッチ

In [ ]:
y_test=y_test.rename(columns={'diagnosis':'test'})

In [ ]:
# スクラッチ
x=np.arange(101)/100
true_rate=[]
false_rate=[]
for i in range(101):
    y_pred=pd.DataFrame((prob>x[i]).astype(int)).rename(columns={0:"pred"})
    pred_test=pd.concat([y_pred,y_test], axis=1)
    tp = sum((pred_test["pred"]==1)&(pred_test["test"]==1))
    fn = sum((pred_test["pred"]==0)&(pred_test["test"]==1))
    fp = sum((pred_test["pred"]==1)&(pred_test["test"]==0))
    tn = sum((pred_test["pred"]==0)&(pred_test["test"]==0))
    true_rate.append(tp/(tp+fn))
    false_rate.append(fp/(fp+tn))

plt.figure(figsize=(8,6))
plt.plot(false_rate,true_rate)
plt.title("ROC curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

## AUC スクラッチ
AUCをライブラリで計算した値と同様の値になりました。

In [ ]:
true_rate1=sorted(true_rate)
false_rate1=sorted(false_rate)
h=true_rate1
w=false_rate1

area=0
for i in range(100):
    area += h[i]*(w[i+1]-w[i])
print('AUC：'+str(area))

## クラス化

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler

class SvmModel():
    
    def __init__(self):
        pass
       
    def model_tuning(self,X_train,X_test,y_train,y_test,tuned_parameters):        
        score = 'f1'
        clf = GridSearchCV(SVC(), #識別器
                           tuned_parameters, #最適化したいパラメータセット
                           scoring= '%s_weighted' % score,
                           cv=5) # 交差検証の回数
        clf.fit(X_train, y_train.as_matrix().reshape(-1,))
        
        dict_params=clf.best_params_

        svm = SVC(gamma=dict_params['gamma'], C=dict_params['C'], probability=True,kernel=dict_params['kernel'])
        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test) 
        y_pred=pd.DataFrame(y_pred)
        y_test=y_test.reset_index().drop("index", axis=1)
        test_pred=pd.concat([y_test, y_pred], axis=1)
        test_pred=test_pred.rename(columns={"diagnosis":"test",0:"pred"})
        
        TP=len(test_pred[(test_pred['test'] == 1) & (test_pred['pred'] == 1)])
        FN=len(test_pred[(test_pred['test'] == 1) & (test_pred['pred'] == 0)])
        FP=len(test_pred[(test_pred['test'] == 0) & (test_pred['pred'] == 1)])
        TN=len(test_pred[(test_pred['test'] == 0) & (test_pred['pred'] == 0)])
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        recall = TP / (TP + FN)
        precision = TP / (TP + FP)
        f1 = 2 * (precision * recall) / (precision + recall)
        
        return 'Bestスコア'+str(clf.best_score_),'Bestパラメータ'+str(clf.best_params_),'Accuracy'+str(accuracy),'Recal'+str(recall),'Precision'+str(precision),'F1'+str(f1)
    
    def roc_plot(self,X_train,X_test, y_train,y_test,C,gamma,kernel):
        
        svm =SVC(kernel=kernel, gamma=gamma, C=C,probability=True).fit(X_train, y_train)
        prob = svm.predict_proba(X_test)[:,1]
        fpr, tpr, thresholds= roc_curve(y_test, prob)

        plt.figure(figsize=(8,6))
        plt.plot(fpr, tpr)
        plt.title("ROC curve")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.show()
        
        return 'AUC:'+ str(np.trapz(tpr,fpr))


df = pd.read_csv("input/bcwd.txt",header=None)    
df_id=df[0] 
df_y=df[1]
mapping={'M':1,'B':0}
df_y=df_y.map(mapping)
df_y=pd.DataFrame(df_y)
df_y.columns = ['diagnosis']
df_x=df.drop([0, 1], axis=1)
from sklearn.preprocessing import StandardScaler
for i in range(2,32):
    x_scaler = StandardScaler()
    x_scaler.fit(df_x[i][:,np.newaxis])
    df_x[i]=x_scaler.transform(df_x[i][:,np.newaxis])
df_x=pd.DataFrame(df_x)
# データ分割
# データ分割
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,test_size=0.2, random_state=0) 
    
tuned_parameters = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
    ]

model=SvmModel()
model. model_tuning(X_train,X_test,y_train,y_test,tuned_parameters)

In [ ]:
model.roc_plot(X_train,X_test, y_train,y_test,100,0.001,'rbf')